In [1]:
import os
import requests
import zipfile
import numpy as np

dataset_url = 'https://github.com/ishaqmarashy/DATALFS/raw/main/JMuBEN.zip'
dataset_dir = './JMuBEN'

# create directory for dataset if it does not exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
# append JMuBEN.zip to the end of the path (this is where we download the file to)
zip_file_path = os.path.join(dataset_dir, 'JMuBEN.zip')

# check if file is downloaded already
if not os.path.exists(zip_file_path):
    # file is not downloaded so fetch the file
    response = requests.get(dataset_url)
    # write file to storage which is recieved from the response
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)
    # unzip to zip file path
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)

# within the concat train and test to become ./JMuBEN/train and JMuBEN ./JMuBEN/test
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

In [2]:
def load_images_and_labels(directory):
    images = []
    labels = []
    # get subdirectories Healthy and Miner
    for class_name in os.listdir(directory):
        # concat subdirectory to get full path
        class_dir = os.path.join(directory, class_name)
        # assign labels using class subdirectory
        # label is determined by filepath
        label = 0 if class_name == 'Miner' else 1
        # append labels and image paths to labels and images respectively
        for filename in os.listdir(class_dir):
            images.append(os.path.join(class_dir, filename))
            labels.append(label)
    return images, labels

# load file directories and their labels
train_images_dir, train_labels = load_images_and_labels(train_dir)
test_images_dir, test_labels = load_images_and_labels(test_dir)

In [3]:
# X_train, X_test, y_train, y_test = train_test_split(train_images_dir, train_labels, test_size=0.2, random_state=42)

# print the number of images and labels
print(f"Train images:{len(train_images_dir)}  Labels:{len(train_labels)}")
print(f"Test images:{len(test_images_dir)}  Labels:{len(test_labels)}")
print(train_images_dir[0], train_labels[0])

Train images:24000  Labels:24000
Test images:6000  Labels:6000
./JMuBEN\train\Healthy\1 (4490).jpg 1


In [5]:
import cv2

def load_normalize_images(image_paths):
    images = []
    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  #  grayscale
        img = cv2.resize(img, (128, 128))  # resize
        img = img / 255.0  # normalize
        images.append(img)
    return np.array(images)

train_images=load_normalize_images(train_images_dir)
test_images=load_normalize_images(test_images_dir)

MemoryError: Unable to allocate 2.93 GiB for an array with shape (24000, 128, 128) and data type float64

# CNN

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

In [ ]:
print(train_images[0])
print(train_labels[0])

plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128, 128, 1)),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'), 
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = tf.nn.relu) ,
    tf.keras.layers.Dense(2,activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(np.asarray(train_images, dtype=np.float32), np.asarray(train_labels), batch_size=128, epochs=6, validation_split=0.2)

In [ ]:
# SGD Classifier used in google collab
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(train_images, train_labels)

In [ ]:
sgd_clf.predict(test_images)